In [ ]:
# !pip install -U langchain-openai
# !pip install openai==0.28
# !pip install chromadb
# !pip install tiktoken

In [ ]:
import os
# setting up the open ai key
os.environ["OPENAI_API_KEY"] = 'open-ai'

In [ ]:
# import necessary libraries
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple
from langchain.chains.question_answering import load_qa_chain

In [ ]:
# Initialize a TextLoader with the path to a text file containing real estate listings.
loader = TextLoader(file_path='realestatelistings.txt')

# Load the documents from the specified file. The result is a list of strings, where each string is a real estate listing.
docs = loader.load()

# Define the model name for the Large Language Model (LLM) that will be used later. In this case, 'gpt-3.5-turbo' is specified.
model_name = 'gpt-3.5-turbo'

# Initialize an instance of the ChatOpenAI model (or LLM) with the specified model name. The temperature is set to 0 for deterministic outputs, and the max_tokens parameter limits the number of tokens (words or pieces of words) the model will generate in a single response.
llm = ChatOpenAI(model_name=model_name, temperature=0, max_tokens=1000)

# Initialize a CharacterTextSplitter with specified chunk size and overlap. This splitter will break down the loaded documents into smaller chunks of 700 characters each, with a 100-character overlap between consecutive chunks. This is useful for processing large documents or for ensuring context is maintained across chunks.
splitter = CharacterTextSplitter(chunk_size=700, chunk_overlap=100)

# Use the splitter to divide the loaded documents into smaller chunks. This results in a list of document chunks, where each chunk is easier to process or embed.
split_docs = splitter.split_documents(docs)

# Initialize an instance of OpenAIEmbeddings. This object is responsible for generating vector embeddings of text using OpenAI's models.
embeddings = OpenAIEmbeddings()

# Create a Chroma vector database from the split documents. This involves generating embeddings for each document chunk using the previously initialized embeddings object, and then storing these embeddings in the database for similarity search and retrieval purposes.
db = Chroma.from_documents(split_docs, embeddings)

In [ ]:
# checking the RAG method

# A string containing the buyer's question or preferences regarding the real estate listing they are interested in.
question = """
    luxuty home in under $3,000,000 in a nice quite neghibourhood. It should have 4 bedroom and 2 bathroom minimum
    """


# A template string that outlines how the LLM should format its response based on the buyer's preferences and the context provided.
# It instructs the LLM to generate a concise, helpful answer based on the information available, without making up answers or paraphrasing,
# and to always end with "thanks for asking!"
template = """Based on the buyer preferences in the context, suggest a home match with useful information about the home.
Make sure you do not paraphrase the reviews, and only use the information provided in the context and buyer preferences.
Don't try to make up an answer.
Use three sentences maximum. Keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}
Question: {question}
Helpful Answer:"""

# A structured representation of the template that specifies the input variables (context and question) and the template itself.
# This is used to dynamically generate prompts for the LLM based on specific inputs.
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template,)

# A boolean flag that determines whether to use a chain helper for retrieval and question answering.
# It's set to False, meaning the else block will be executed to handle the query without using the chain helper.
use_chain_helper = False

# Conditional logic to determine the query processing approach based on the value of use_chain_helper.
if use_chain_helper:
    # If true, it initializes a RetrievalQA instance with specific parameters, including the LLM, a retriever based on the vector database, and the custom prompt template.
    # This approach would use a more complex chain of retrieval and question answering but is skipped due to the flag being False.
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
    print(rag.run(question))
else:
    # Since use_chain_helper is False, this block executes a simpler process:
    # 1. Performs a similarity search in the vector database to find documents (listings) that are semantically close to the buyer's query.
    # 2. Initializes a QA chain with the LLM and the custom prompt template.
    # 3. Runs the QA chain with the similar documents and the buyer's question to generate a response.
    similar_docs = db.similarity_search(question, k=3)
    chain = load_qa_chain(llm, prompt = QA_CHAIN_PROMPT, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, question = question))


This exquisite 4-bedroom, 2-bathroom home in the prestigious neighborhood of Kensington is the perfect luxury home for you. With its regal charm, elegant facade, and period features, this property offers a taste of luxury living. The spacious bedrooms, beautifully appointed bathrooms, and grand living areas provide a comfortable retreat and are perfect for entertaining. The fully equipped modern kitchen and picturesque garden complete this stately residence. Thanks for asking!


In [ ]:
# Collect buyer preferences, such as the number of bedrooms, bathrooms, location,
# and other specific requirements from a set of questions or telling the buyer to enter their preferences in natural language.
personal_questions = [
                "How big do you want your house to be?"
                "What are 3 most important things for you in choosing this property?",
                "Which amenities would you like?",
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",
            ]

personal_answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

In [ ]:
# Initializes the Large Language Model (LLM) with specific configurations.
model_name = "gpt-3.5-turbo"  # The model version to be used.
temperature = 0.0  # The temperature setting for deterministic outputs.
llm = ChatOpenAI(model_name=model_name, temperature=0, max_tokens=1000)  # Instantiates the LLM with a maximum token limit.

# Prepares a chat history to simulate a conversation between the user and AI.
history = ChatMessageHistory()
# Adds an initial user message to the chat history to set the context of the AI's role.
history.add_user_message(f"""You are AI that will recommend user a home match based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
# Iterates through the series of personal questions and answers, adding them to the chat history.
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])  # AI asks a question.
    history.add_user_message(personal_answers[i])  # Simulated user response.

# Adds a final AI message prompting for a summary of a home to consider for recommendation.
history.add_ai_message("""Now tell me a home summary of a home you're considering recommending, and specify how you want me to respond to you with the final recommendation.""")

# Initializes a memory system to summarize conversations, focusing on the home recommendation based on the user's answers.
summary_memory = ConversationSummaryMemory(
    llm=llm,  # The LLM instance.
    memory_key="recommendation_summary",  # A key to identify this particular memory.
    input_key="input",  # Specifies the key for input data in the memory system.
    buffer=f"The human answered {len(personal_questions)} personal questions). Use them to clealy describe the different aspects of home.",  # A buffer message summarizing the conversation's context.
    return_messages=True)  # Indicates that the memory system should return messages.

# Defines a custom memory class to save additional context from the Q&A session into the conversation history.
class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        # Extracts input and output text for storage.
        input_str, output_str = self._get_input_output(inputs, outputs)
        # Adds the output (AI's response) to the chat history.
        self.chat_memory.add_ai_message(output_str)

# Initializes the custom memory to store Q&A context.
conversational_memory = MementoBufferMemory(
    chat_memory=history,  # The chat history instance.
    memory_key="questions_and_answers",  # A key to identify this part of the memory.
    input_key="input"  # Specifies the key for input data in this memory system.
)

# Combines the two memory systems into a single entity for comprehensive context management.
memory = CombinedMemory(memories=[conversational_memory, summary_memory])


In [ ]:
# Defines a template for the conversation between the human user and the AI recommender.
# The template sets the context for the AI's role as a Home Match Recommender and structures the conversation output,
# including the summary of recommendations and a log of personal questions and answers.
RECOMMENDER_TEMPLATE = """The following is a friendly conversation between a human and an AI Home Match Recommender.
                        The AI follows human instructions and provides home recommendation for a human buyer based on the buyer preferences
                        and human's persona derived from their answers to questions.

Summary of Recommendations:
{recommendation_summary}
Personal Questions and Answers:
{questions_and_answers}
Human: {input}
AI:"""

# Initializes a prompt template with specified input variables. These variables are placeholders that will be filled with
# actual data during the execution of the conversation chain. The `RECOMMENDER_TEMPLATE` serves as the structure for the conversation.
PROMPT = PromptTemplate(
    input_variables=["recommendation_summary", "input", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)

# Creates an instance of ConversationChain. This is a mechanism that uses the provided Large Language Model (llm),
# the memory system (memory), and the structured prompt (PROMPT) to generate AI responses within a conversational context.
# The `verbose` parameter is set to True to enable detailed logging or output, which can be useful for debugging or understanding the AI's decision-making process.
recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)


In [ ]:

plot_rating_instructions = f"""
        =====================================
    === START HOME MATCH PLOT SUMMARY FOR {question} ===
    THe below found documents which is similar to the user preference.
    {db.similarity_search(question, k=3)}
    === END MOVIE PLOT SUMMARY ===
    =====================================

    RATING INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
    AI will provide a highly personalized Home Recommendation based only on the buyer preferences summary human provided
    and human answers to questions included with the context.
    AI should be very sensible to human personal preferences captured in the answers to personal questions,
    and should not be influenced by anything else.
    AI will also build a persona for human based on human answers to questions, and use this persona to rate the home match.

    OUTPUT FORMAT:
    First, include that persona you came up with in the home recommendation. Describe the persona in a few sentences.
    Explain how human preferences captured in the answers to personal questions influenced creation of this persona.
    In addition, consider other similar features of home for this human that you might have as they might give you more information about human's preferences.
    Your goal is to provide a home match recommendation that is as close as possible to the buyer preferences.
    Remember that human buys home only few times in lifetime and wants to have their perfect home, so your recommendation should be as accurate as possible.
    You will include a logical explanation for your recommendation based on human persona you've build and human responses to questions.
    YOUR REVIEW MUST END WITH TEXT: Thank you for your Query
    FOLLOW THE INSTRUCTIONS STRICTLY, OTHERWISE HUMAN WILL NOT BE ABLE TO UNDERSTAND YOUR RECOMMENDATION.
"""


# Uses the `recommender` instance (a ConversationChain object) to make a prediction based on a given input.
# This input, `plot_rating_instructions`, is presumably a string containing instructions or criteria that guide the AI
# in how to analyze and rate potential home matches based on the user's preferences.
prediction = recommender.predict(input=plot_rating_instructions)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Home Match Recommender.
                        The AI follows human instructions and provides home recommendation for a human buyer based on the buyer preferences
                        and human's persona derived from their answers to questions.

Summary of Recommendations:
[SystemMessage(content='The human answered 4 personal questions). Use them to clealy describe the different aspects of home.')]
Personal Questions and Answers:
Human: You are AI that will recommend user a home match based on their answers to personal questions. Ask user 4 questions
AI: How big do you want your house to be?What are 3 most important things for you in choosing this property?
Human: A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
AI: Which amenities would you like?
Human: A quiet neighborhood, good local schools, and convenient shopp

In [ ]:
# `final_recommendation` is a string that provides a new set of instructions for the AI.
# This string outlines the task for the AI: to recommend the most suitable home to the user.
# It specifies that the AI should explain why this particular home is the best match for the user,
# focusing on reasons that align with the user's preferences, without including any other recommendations in the explanation.
final_recommendation = """ Now that AI has understood user preferences, AI will recommend a home match to human the one that human will like the most.
                            AI will respond with home recommendation, and short explanation for why human will like it over all other homes.
                            AI will not include any recommendation in your explanation, only the reasons why human will like it the most.
                            However, the home you will pick must be one of the homes you find most suitable.

                            PROVIDE THE FINAL OUTCOME AS FOLLOWING:
                            FIRST DESCRIBE WHAT YOU HAVE FOUND BASED ON THE USER PREFERENCES. \n
                            THEN PROVIDE - DETAILS OF THE HOME SUCH AS LOCATION, BEDROOM, PRICE, LOCATION, \n
                            THEN PROVIDE - PRICE AND COMPARE IT WITH THE USERS ASKING PRICE IF IT IS GIVEN \n

                            EXAMPLE.
                            "
                            Based on your preferences for a comfortable three-bedroom house with a spacious kitchen and a cozy living room, a quiet neighborhood with good local schools and convenient shopping options, and easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads,
                            I recommend a luxury home in the Kensington neighborhood.

                            This home offers the perfect combination of comfort and luxury with its spacious bedrooms, elegant features, and fully equipped modern kitchen.
                            The neighborhood itself is known for its upscale vibe, high-end shopping, gourmet restaurants, and cultural attractions. Additionally, the proximity to Hyde Park allows for outdoor activities and relaxation in the heart of the city.

                            Overall, this home in the Kensington neighborhood aligns perfectly with your preferences and offers a luxurious living experience in a desirable location.
                            "
                            """

# The recommender is again used to predict the output, this time using the `final_recommendation` instructions.
# This prompts the AI to generate a response that includes a specific home recommendation along with a justification tailored to the user's preferences.
prediction = recommender.predict(input=final_recommendation)

# Prints the output of the prediction, which should be the AI's recommended home and the reasons it believes the home is the best match for the user.
print(prediction)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Home Match Recommender.
                        The AI follows human instructions and provides home recommendation for a human buyer based on the buyer preferences
                        and human's persona derived from their answers to questions.

Summary of Recommendations:
[SystemMessage(content="The AI provides a highly personalized home recommendation based on the human's preferences. The human is looking for a luxury home in a nice quiet neighborhood with four bedrooms and two bathrooms, priced under $3,000,000. The AI suggests a property in the prestigious neighborhood of Kensington, which offers a taste of luxury living. The house has a spacious kitchen, cozy living room, and picturesque garden. The neighborhood is known for its upscale vibe, with high-end shopping, gourmet restaurants, and cultural attractions. It also provides easy access to